### A no-nonsense guide to generators!!

Adapted from: Iterators and Generators by Aaron Maxwell (aaron@powerfulpython.com)

In [67]:
def gen_squares(max_root):
    for x in range(max_root):
        yield x**2
squares = gen_squares(5)
for sq in squares:
    print(sq)

0
1
4
9
16


##### key points:
- type of `squares` : generator object
- type of `gen_squares` : generator_function


In [68]:
type(squares)

generator

Some notes about generators:
- The scalability benefit of the generator object is like a raw iterator
- every generator object is an iterator but not the other way around
- easiest way to create an iterator is the generator object

In [69]:
#### The next() thing

In [70]:
squares = gen_squares(5)
print(next(squares))
print(next(squares))
print(next(squares))
print(next(squares))
print(next(squares))
if next(squares, None) == None: # lets you supply the default value
    print('thats all folks')

0
1
4
9
16
thats all folks


In [71]:
def gen_up_to(limit):
    n = 0
    while n <= limit:
        yield n
        n += 1
limit = 10
it = gen_up_to(limit)

In [72]:
next(it, None)

0

### Use case 1 : `pattern matching` log files with `small memory footprint`

In [73]:
import sys
def matching_lines(pattern, path):
    '''
    generator function matches lines from a log file based on pattern
    :param path : file with records one per line
    :return yields an object
    '''
    with open(path) as handler_list:
        for line in handler_list:
            if pattern in line:
                yield line.rstrip('\n')
pattern, path = 'WARNING', 'log.txt'

In [74]:
print([line for line in matching_lines(pattern, path)])

['WARNING : Disk usage exceeding 85%', 'WARNING : Almost out of coconut water']


### Use case 2 : `Transforming` log files to `dictionaries`

In [75]:
def parse_log_records(lines):
    '''
    return a dictionary to parse log records
    : param lines
    : yield dict with two keys
    '''
    for line in lines:
        level, message = line.split(": ", 1)
        yield {"level" : level, "message" : message}

In [76]:
print([record for record in parse_log_records(matching_lines(pattern, path))])

[{'level': 'WARNING ', 'message': 'Disk usage exceeding 85%'}, {'level': 'WARNING ', 'message': 'Almost out of coconut water'}]


##### With the two building blocks `parse_log_records` and `matching_lines`, lets proceed to build a class `Logs`, which comes quite handy during data engineering.

In [77]:
class Logs:
    def __init__(self, log_file_path):
        self.log_file_path = log_file_path
    def records(self):
        with open(self.log_file_path) as log_lines:
                 for record in parse_log_records(log_lines):
                    yield record
    def warnings(self):
        log_lines = matching_lines("WARNING", self.log_file_path)
        for record in parse_log_records(log_lines):
            yield record

In [78]:
logs = Logs('log.txt')

In [80]:
print([record for record in logs.records()])

[{'level': 'WARNING ', 'message': 'Disk usage exceeding 85%\n'}, {'level': 'DEBUG ', 'message': "User 'tinytim' upgraded to pro\n"}, {'level': 'INFO ', 'message': 'Sent email campaign, completed normally\n'}, {'level': 'WARNING ', 'message': 'Almost out of coconut water\n'}]
